In [1]:
import praw
import pandas as pd
import datetime
from praw.models import MoreComments
import nltk
import nltk
nltk.download('vader_lexicon')
import nltk.sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Reddit API authentication:

In [2]:
reddit = praw.Reddit(client_id='BxrF2g9d3egQ4g', \
                     client_secret='T-pkCmWCBxLjVlNZk88OjvcH7sk', \
                     user_agent='SDS2019')

### Download comments for movies on reddit with timestamp

In [3]:
def get_reddit_comments(movie_title, testing = True):
    subreddit = reddit.subreddit('movies')
    
    comments = []
    subid = []
    treeid = []
    date = []
    
    searchresults = subreddit.search(movie_title, sort = 'top')
    i = 0
    for subidx, submission in enumerate(searchresults):
        
        if testing:
            if i > 5:
                break
            i += 1
            
        try:
            comment_tree = submission.comments.list()
            for treeidx, top_lvl_comment in enumerate(comment_tree):
                
                # Erstat disse to linjer hvis i vil have 
                # data ud af "morecomments" objekterne
                if isinstance(top_lvl_comment, MoreComments):
                    continue
                
                subid.append(subidx)
                comments.append(top_lvl_comment.body)
                treeid.append(treeidx)
                date.append(top_lvl_comment.created_utc)
                
        except:
            pass
    df = pd.DataFrame({'submission': subid, 'comment_tree': treeid, 'comment': comments, 'date': date})
    df['date'] = pd.to_datetime(df['date'], unit = 's')
    df = df.set_index(['date'])
    df = df.sort_index()
    
    #df_before = df[:release_date - datetime.timedelta(days = 1)]
    #df_after = df[release_date:release_date + datetime.timedelta(days = 60)]
    
    return df

In [4]:
##### TESTING STUFF #####
#release_date = datetime.date(year = 2019, month = 7, day = 26)
#df = get_reddit_comments('once upon a time in hollywood', release_date, testing = True)
#print(start_date)

In [5]:
##### TESTING STUFF #####
#print(once_upon)
#pd.options.display.max_colwidth = 3000
#once_upon['comment'].iloc[[3]]

### Linguistic analysis of commments:

In [6]:
def sentiment(movie_df):
    vader = nltk.sentiment.vader.SentimentIntensityAnalyzer()
    vader_df = pd.DataFrame(list(movie_df['comment'].apply(vader.polarity_scores)))
    vader_mean = vader_df['compound'].mean()
    return vader_mean, len(vader_df)

### Grabbing movies and release dates from movie dataframe

In [7]:
df = pd.read_pickle('full.pkl')
df = df[df['year'] >= 2013]

movie_list = df['title'].tolist()
date_list = df['releaseDate'].tolist()

### Getting sentiment scores for all movies

In [ ]:
#movie_list = ['avengers endgame', 'captain marvel', 'hobbs & shaw']
#date_list = [datetime.date(2019, 4, 26), datetime.date(2019, 3, 8), datetime.date(2019, 8, 2)]

sentiments_before = []
sentiments_after = []
num_comments_before = []
num_comments_after = []
i = 1
for movie, date in zip(movie_list, date_list):
    try:
        comments = get_reddit_comments(movie, testing = False)
    except:
        sentiments_before.append(None)
        sentiments_after.append(None)
        num_comments_before.append(None)
        num_comments_after.append(None)
        continue
    
    try:
        comments_before = comments[:date - datetime.timedelta(days = 1)]
    except:
        pass
        
    try:
        comments_after = comments[date:date + datetime.timedelta(days = 60)]
    except:
        pass
    
    try:
        snt_before = sentiment(comments_before)
    except:
        pass
        
    try:
        snt_after = sentiment(comments_after)
    except:
        pass
    
    try:
        sentiments_before.append(snt_before[0])
    except:
        sentiments_before.append(None)
        
    try:
        sentiments_after.append(snt_after[0])
    except:
        sentiments_after.append(None)
        

    try:
        num_comments_before.append(snt_before[1])
    except:
        num_comments_before.append(None)
        
    try:
        num_comments_after.append(snt_after[1])
    except:
        num_comments_after.append(None)
        
    print(i)
    i += 1

df['sentimentBefore'] = sentiments_before
df['sentimentAfter'] = sentiments_after
df['numCommentsBefore'] = num_comments_before
df['numCommentsAfter'] = num_comments_after

df.to_pickle('full_with_sentiment.pkl')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [ ]:
print(sentiments_before)
print(sentiments_after)
print(num_comments_before)
print(num_comments_after)